In [1]:
import numpy as np

import nibabel as nib

from utils import read_tracking, write_tracking

In [2]:
leg_paths = {
    "data": "leg/leg_data.nii.gz",
    "conf": "leg/leg_conf.nii.gz",
    "tracking": "leg/leg_tracking.csv",
}

liver_paths = {
    "data": "liver/liver_all_sweeps_images.nii.gz",
    "conf": "liver/liver_all_sweeps_confidence_maps.nii.gz",
    "tracking": "liver/liver_all_sweeps_tracking.csv",
}

spine_paths = {
    "data": "spine/spine_all_sweeps.nii.gz",
    "conf": "spine/spine_all_sweeps_conf.nii.gz",
    "tracking": "spine/spine_all_sweeps_tracking.csv",
}

In [3]:
leg = {
    "data": nib.load(leg_paths["data"]).get_fdata(),
    "conf": nib.load(leg_paths["conf"]).get_fdata(),
    "tracking": read_tracking(leg_paths["tracking"]),
}

liver = {
    "data": nib.load(liver_paths["data"]).get_fdata(),
    "conf": nib.load(liver_paths["conf"]).get_fdata(),
    "tracking": read_tracking(liver_paths["tracking"]),
}

spine = {
    "data": nib.load(spine_paths["data"]).get_fdata(),
    "conf": nib.load(spine_paths["conf"]).get_fdata(),
    "tracking": read_tracking(spine_paths["tracking"]),
}

In [4]:
# Print all shapes

print("Leg data shape:", leg["data"].shape)
print("Leg confidence shape:", leg["conf"].shape)
print("Leg tracking shape:", leg["tracking"].shape)

print("Liver data shape:", liver["data"].shape)
print("Liver confidence shape:", liver["conf"].shape)
print("Liver tracking shape:", liver["tracking"].shape)

print("Spine data shape:", spine["data"].shape)
print("Spine confidence shape:", spine["conf"].shape)
print("Spine tracking shape:", spine["tracking"].shape)

Leg data shape: (221, 537, 1714)
Leg confidence shape: (221, 537, 1714)
Leg tracking shape: (1714, 4, 4)
Liver data shape: (1200, 512, 256)
Liver confidence shape: (1200, 512, 256)
Liver tracking shape: (1200, 4, 4)
Spine data shape: (164, 1300, 1341)
Spine confidence shape: (164, 1300, 1341)
Spine tracking shape: (1341, 4, 4)


In [5]:
# Liver is in reverse order, we are going to fix that, if that is not the case for you comment this out

liver["data"] = np.swapaxes(liver["data"], 0, 1)
liver["data"] = np.swapaxes(liver["data"], 1, 2)
liver["data"] = np.swapaxes(liver["data"], 0, 1)

liver["conf"] = np.swapaxes(liver["conf"], 0, 1)
liver["conf"] = np.swapaxes(liver["conf"], 1, 2)
liver["conf"] = np.swapaxes(liver["conf"], 0, 1)

In [6]:
# Print all shapes

print("Leg data shape:", leg["data"].shape)
print("Leg confidence shape:", leg["conf"].shape)
print("Leg tracking shape:", leg["tracking"].shape)

print("Liver data shape:", liver["data"].shape)
print("Liver confidence shape:", liver["conf"].shape)
print("Liver tracking shape:", liver["tracking"].shape)

print("Spine data shape:", spine["data"].shape)
print("Spine confidence shape:", spine["conf"].shape)
print("Spine tracking shape:", spine["tracking"].shape)

Leg data shape: (221, 537, 1714)
Leg confidence shape: (221, 537, 1714)
Leg tracking shape: (1714, 4, 4)
Liver data shape: (256, 512, 1200)
Liver confidence shape: (256, 512, 1200)
Liver tracking shape: (1200, 4, 4)
Spine data shape: (164, 1300, 1341)
Spine confidence shape: (164, 1300, 1341)
Spine tracking shape: (1341, 4, 4)


In [7]:
# Write min max and type of each array

for d in [leg, liver, spine]:

    for k in ["data", "conf"]:

        # if float and max is close to 255, convert to uint8
        # if float and max is close to 1.0, multiply by 255 and convert to uint8

        if d[k].dtype == np.float64 or d[k].dtype == np.float32:

            if np.isclose(np.max(d[k]), 255.0):
                d[k] = d[k].astype(np.uint8)
            elif np.isclose(np.max(d[k]), 1.0):
                d[k] = (d[k] * 255).astype(np.uint8)

In [8]:
# Print type and min max values

for d in [leg, liver, spine]:
    
        for k in ["data", "conf"]:
    
            print(k, d[k].dtype, np.min(d[k]), np.max(d[k]))

data uint8 0 255
conf uint8 17 255
data uint8 0 255
conf uint8 5 255
data uint8 0 255
conf uint8 0 255


In [11]:
# pixdims
leg_pixdim = [1., 0.172336, 0.172780, 1., 1., 1., 1., 1.]
liver_pixdim = [1., 0.3125, 0.271318, 1., 1., 1., 1., 1.]
spine_pixdim = [1., 0.228264, 0.076923, 1., 1., 1., 1., 1.]

In [12]:
# Save data and conf to same directories for each, add refined to the name

for d, p, pixdim in zip([leg, liver, spine], [leg_paths, liver_paths, spine_paths], [leg_pixdim, liver_pixdim, spine_pixdim]):

    nii_data = nib.Nifti1Image(d["data"], np.eye(4))
    nii_data.header["pixdim"] = pixdim
    nib.save(nii_data, p["data"].replace(".nii.gz", "_refined.nii.gz"))

    nii_conf = nib.Nifti1Image(d["conf"], np.eye(4))
    nii_conf.header["pixdim"] = pixdim
    nib.save(nii_conf, p["conf"].replace(".nii.gz", "_refined.nii.gz"))

In [13]:
print(nii_data.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [   3  164 1300 1341    1    1    1    1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1.       0.228264 0.076923 1.       1.       1.       1.       1.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : 0.0
qoffset_y       : 0.0
qoffset_z       : 0.0
sr